# Condiciones sociodemográficas y la percepción de la seguridad pública en México


## Construcción (no se tiene que volver a correr)

### Datos a nivel individual

In [1]:
#!pip install simpledbf
#!pip install INEGIpy

In [5]:
import pandas as pd
import numpy as np
from simpledbf import Dbf5
import re

PyTables is not installed. No support for HDF output.
SQLalchemy is not installed. No support for SQL output.


In [3]:
# lectura de las bases de datos del a ENVIPE
base_path = './bases/'
paths = ['{}envipe_20{}/'.format(base_path, str(i)) for i in range(15,22)]

In [25]:
# sirve para estandarizar el nombre de las bases y poder loopear
# solo se tiene que correr una vez
import os

for path in paths:
  files = os.listdir(path)
  for file in files:
    os.rename("{}{}".format(path, file), "{}{}".format(path, file.lower()))

In [ ]:
viviendas_lst = list()
hogares_lst = list()
sociodem_lst = list()
victim_lst = list()
for path in paths:
    fecha = re.findall('\d\d\d\d', path)[0]
    viv = Dbf5(path + 'tvivienda.dbf', codec='latin1').to_dataframe()
    hog = Dbf5(path + 'thogar.dbf', codec='latin1').to_dataframe()
    socdem = Dbf5(path + 'tsdem.dbf', codec='latin1').to_dataframe()
    try: vict = Dbf5(path + 'tper_vic.dbf', codec='latin1').to_dataframe()
    except: vict = Dbf5(path + 'tper_vic1.dbf', codec='latin1').to_dataframe()
    viv['año'] = fecha
    hog['año'] = fecha
    socdem['año'] = fecha
    vict['año'] = fecha
    viviendas_lst.append(viv)
    sociodem_lst.append(socdem)
    victim_lst.append(vict)

In [71]:
# abre las bases y concatena en un solo DF para todos los años

def __concat_tablas(file_name, paths):
    dfs = list()
    for path in paths:
        fecha = re.findall('\d\d\d\d', path)[0]
        df = Dbf5(path + file_name, codec='latin1').to_dataframe()
        df['año'] = fecha
        dfs.append(df)
    final_df = pd.concat(dfs, axis=0)
    return final_df, dfs

In [74]:
viviendas, viv_lst = __concat_tablas('tvivienda.dbf',paths)
hogares, hog_lst = __concat_tablas('thogar.dbf',paths)
sociodem, sdem_lst = __concat_tablas('tsdem.dbf',paths)
victim, vict_lst = __concat_tablas('tper_vic1.dbf',paths)

###### Tabla de viviendas

In [75]:
viviendas.columns
# llaves: viv_sel y ump

Index(['ID_VIV', 'UPM', 'VIV_SEL', 'ENT', 'MUN', 'AREAM', 'RESUL_V', 'AP1_1',
       'AP1_2', 'AP2_1', 'AP2_2', 'AP2_3', 'NAP2_3', 'FAC_VIV', 'EST_DIS',
       'UPM_DIS', 'DOMINIO', 'FAC_VIV_AM', 'año', 'RESULT_V', 'CVE_ENT',
       'NOM_ENT', 'CVE_MUN', 'NOM_MUN', 'PER'],
      dtype='object')

In [76]:
viviendas['cve_agee'] = np.select([viviendas.CVE_ENT.isna()], [viviendas.ENT], default = viviendas.CVE_ENT)
viviendas['resultado'] = np.select([viviendas.RESULT_V.isna()], [viviendas.RESUL_V], default = viviendas.RESULT_V)
viviendas = viviendas[['VIV_SEL','UPM','cve_agee','AP1_1','AP1_2','resultado','año']]

###### Tabla de hogares

In [78]:
hogares.columns
# llaves viv_sel, upm y hogar

Index(['ID_HOG', 'UPM', 'VIV_SEL', 'HOGAR', 'RESUL_H', 'N_INF', 'R_SEL',
       'TOT_PER', 'FAC_HOG', 'EST_DIS', 'UPM_DIS', 'DOMINIO', 'FAC_HOG_AM',
       'año', 'ID_VIV', 'CVE_ENT', 'NOM_ENT', 'CVE_MUN', 'NOM_MUN'],
      dtype='object')

In [104]:
hogares = hogares[['HOGAR','VIV_SEL','UPM','año','RESUL_H']]

###### Tabla sociodemográfica

In [111]:
sociodem.columns
# llaves n_ren, hogar, viv_sel, upm

Index(['ID_PER', 'UPM', 'VIV_SEL', 'HOGAR', 'RESUL_H', 'N_REN', 'PAREN',
       'SEXO', 'EDAD', 'FN_DIA', 'FN_MES', 'CODIGO', 'NIV', 'GRA', 'AP3_8',
       'AP3_9', 'AP3_10', 'FAC_HOG', 'EST_DIS', 'UPM_DIS', 'DOMINIO',
       'FAC_HOG_AM', 'año', 'ID_VIV', 'ID_HOG', 'CVE_ENT', 'NOM_ENT',
       'CVE_MUN', 'NOM_MUN'],
      dtype='object')

In [113]:
sociodem = sociodem[['N_REN','HOGAR','VIV_SEL','UPM','SEXO','EDAD','NIV','AP3_8','año']]

###### Tabla percepción de seguridad

In [116]:
victim.columns

Index(['ID_PER', 'UPM', 'VIV_SEL', 'HOGAR', 'RESUL_H', 'R_SEL', 'AREAM',
       'AP4_1', 'AP4_2_01', 'AP4_2_02',
       ...
       'NOM_MUN', 'AP4_2_13', 'AP4_5_17', 'AP4_5_18', 'AP4_10_16', 'AP5_3_11',
       'AP5_4_11', 'AP5_5_11', 'AP5_6_11', 'AP4_4_A'],
      dtype='object', length=196)

In [117]:
victim = victim[['R_SEL','HOGAR','VIV_SEL','UPM','AP4_2_01','AP4_2_02','AP4_2_03','AP4_2_04','AP4_2_05',
                 'AP4_2_07','AP4_2_08','AP4_2_11','AP4_3_1','AP4_8_1','AP4_8_2','AP4_8_3','año']]

###### cambio de nombres y guardado

In [118]:
viviendas.columns = ['viv_sel','upm','cve_agee','tipo_vivienda','barrera_vivienda','result_v','año']
hogares.columns = ['hogar','viv_sel','upm','año','result_h']
sociodem.columns = ['renglon','hogar','viv_sel','upm','sexo','edad','escolaridad','ocupacion','año']
victim.columns = ['renglon','hogar','viv_sel','upm','pobreza','desempleo','narco','precios','inseguridad','agua',
                  'corrupcion','impunidad','perc_seguridad','falta_alumbrado','falta_agua','baches_fugas','año']

###### base individual

In [155]:
# las preguntas sobre las tres cosas que más le preocupan al encuestado la reducimos a dos categorías para facilitar
# el modelo e interpretación. Las categorías son: 
# bienestar económico: le preocupa al menos una de las variables que tienen que ver con condiciones económicas y materiales
# seguridad: le preocupa al menos una de las variables que tienen que ver con crimen o seguridad pública

victim['bienestar_econ'] = np.select([victim.pobreza.eq('1') | victim.desempleo.eq('1') | victim.precios.eq('1') | victim.agua.eq('1')], [1])
victim['seguridad'] = np.select([victim.narco.eq('1') | victim.inseguridad.eq('1') | victim.corrupcion.eq('1') | victim.impunidad.eq('1')], [1])

In [160]:
victim = victim[['renglon','hogar','viv_sel','upm','bienestar_econ','seguridad','falta_alumbrado',
                 'falta_agua','baches_fugas','perc_seguridad','año']]

In [161]:
# construcción de base final a nivel encuestado
# hacemos merge de las tablas usando las llaves definidas
base_individual = hogares.merge(viviendas, 'left')

In [168]:
base_individual = sociodem.merge(base_individual, 'left')

In [172]:
base_individual = victim.merge(base_individual, 'left')

In [182]:
# filtrado por:
# result_v: 01 y 02 son entrevistas completas
# result_h: A y B son entrevistas completas
# edad: limitamos las observaciones a las de personas mayores a los 14 años 

base_individual['edad'] = base_individual.edad.astype(int)
base_final_ind = base_individual[(base_individual.result_v.isin(['01','02'])) & 
                                 (base_individual.result_h.isin(['A','B'])) & 
                                 (base_individual.edad > 14)]

In [183]:
base_final_ind.shape

(628092, 20)

In [9]:
base_final_ind[base_final_ind.año.between(2016,2020)].shape

(451626, 20)

In [13]:
base_final_ind.loc[base_final_ind.falta_alumbrado == 2, 'falta_alumbrado'] = 0 # 1: hay problema, 0: no hay problema
base_final_ind.loc[base_final_ind.falta_agua == 2, 'falta_agua'] = 0 # 1: hay problema, 0: no hay problema
base_final_ind.loc[base_final_ind.baches_fugas == 2, 'baches_fugas'] = 0 # 1: hay problema, 0: no hay problema
base_final_ind.loc[base_final_ind.perc_seguridad == 2, 'perc_seguridad'] = 0 # 1: seguro, 0: no seguro (nivel colonia)
base_final_ind.loc[base_final_ind.sexo == 2, 'sexo'] = 0 # 1: masculino, 0: femeninio
base_final_ind.loc[base_final_ind.barrera_vivienda == 2, 'barrera_vivienda'] = 0  # 1: hay barrera, 0: no hay barrera

In [17]:
dummies = pd.get_dummies(base_final_ind.tipo_vivienda)
cols = ['casa_independiente','departamento','vecindad','cuarto_azotea','local_no_habitacional']
dummies.columns = cols
base_final_ind[cols] = dummies

In [20]:
# tiene trabajo aunque no haya trabajado en la semana
base_final_ind['con_trabajo'] = np.select([base_final_ind.ocupacion.isin([1,2])], [1]) 
# no tiene trabajo por estar incapacitado, pensionado u ocupado en estudios/trabajos del hogar
base_final_ind['no_puede_trabajar'] = np.select([base_final_ind.ocupacion.isin([4,5,6,7])], [1]) 
# buscó trabajo o no trabaja
base_final_ind['no_trabajo'] = np.select([base_final_ind.ocupacion.isin([3,8])], [1])

In [23]:
base_final_ind = base_final_ind[base_final_ind.ocupacion != 9]

In [27]:
# estructura de la base individual: [ llaves (incluyendo año) , Y, X]
base_final_ind = base_final_ind[['año','cve_agee','renglon','hogar','viv_sel','upm','perc_seguridad','bienestar_econ','seguridad','falta_alumbrado','falta_agua','baches_fugas',
                                 'sexo','edad','escolaridad','barrera_vivienda','casa_independiente','departamento','vecindad','cuarto_azotea','local_no_habitacional',
                                 'con_trabajo','no_puede_trabajar','no_trabajo']]

In [29]:
base_final_ind.to_csv('base_individual.csv', index=False)

### Datos a nivel Entidad Federativa

#### Datos del BIE o BISE

In [1]:
from INEGIpy import MarcoGeoestadistico, Indicadores

token = '92170321-528f-f1dd-5d59-f8613e072746'
inegi = Indicadores(token)
marco = MarcoGeoestadistico()
entidades = marco.Entidades()

In [4]:
# Tasa de desocupación
indicadores = ['6200093973','6200093960'] # indicadores para población desocupada y población económicamente activa
nombres = ['pob_desocupada','pob_econ_act'] 
dfs = []
for ent in entidades.cve_agee:
    df = inegi.obtener_df(indicadores, nombres, ent) 
    df['desocupacion_{}'.format(ent)] = df.pob_desocupada / df.pob_econ_act
    df = df.drop(nombres, axis=1)
    dfs.append(df)
desocupacion = pd.concat(dfs, axis=1)
desocupacion.columns = entidades.nom_agee
desocupacion.head()

nom_agee,Aguascalientes,Baja California,Baja California Sur,Campeche,Coahuila de Zaragoza,Colima,Chiapas,Chihuahua,Ciudad de México,Durango,...,Quintana Roo,San Luis Potosí,Sinaloa,Sonora,Tabasco,Tamaulipas,Tlaxcala,Veracruz de Ignacio de la Llave,Yucatán,Zacatecas
fechas,,,,,,,,,,,,,,,,,,,,,
2005-03-01,0.045985,0.016711,0.026297,0.024886,0.049947,0.031496,0.021792,0.034776,0.055555,0.041806,...,0.027402,0.032835,0.031037,0.036170,0.036384,0.051734,0.038803,0.032398,0.026561,0.030945
2005-06-01,0.052603,0.011904,0.023395,0.027898,0.046969,0.025865,0.020889,0.020859,0.053982,0.034235,...,0.026357,0.024953,0.032849,0.032138,0.028803,0.039959,0.044005,0.022768,0.023858,0.024025
2005-09-01,0.053979,0.012861,0.025040,0.027191,0.043184,0.032981,0.023998,0.024069,0.063028,0.033595,...,0.026833,0.027851,0.036881,0.041499,0.037307,0.047872,0.048996,0.017295,0.026917,0.034875
2005-12-01,0.041498,0.015517,0.017595,0.018431,0.040355,0.032071,0.017049,0.022528,0.048969,0.029530,...,0.026530,0.022444,0.029548,0.028885,0.029688,0.028912,0.041138,0.022288,0.025186,0.022758
2006-03-01,0.049753,0.015562,0.023115,0.021095,0.058409,0.036304,0.018265,0.028837,0.054556,0.040251,...,0.024710,0.024619,0.031025,0.028323,0.036484,0.038939,0.058176,0.024226,0.022553,0.037320


In [5]:
# ITAEE
indicadores = np.arange(start = 491909, stop = 491909 + 7*32, step = 7).astype(str) # indicadores de los ITAEE (de 491909 sube en 7 para cada entidad)
itaee = inegi.obtener_df(indicadores, entidades.nom_agee)
itaee.head()

nom_agee,Aguascalientes,Baja California,Baja California Sur,Campeche,Coahuila de Zaragoza,Colima,Chiapas,Chihuahua,Ciudad de México,Durango,...,Quintana Roo,San Luis Potosí,Sinaloa,Sonora,Tabasco,Tamaulipas,Tlaxcala,Veracruz de Ignacio de la Llave,Yucatán,Zacatecas
fechas,,,,,,,,,,,,,,,,,,,,,
1980-03-01,28.824944,48.099253,33.440643,74.083743,39.803476,43.462187,55.666190,39.645739,49.415928,47.098602,...,28.969635,40.895667,54.446501,42.474677,46.162374,42.042124,53.556819,51.915467,39.536187,34.981667
1980-06-01,28.833592,48.248054,33.689560,73.917576,39.845656,43.518825,55.793318,39.834734,49.958848,47.156999,...,29.157197,41.127703,54.068383,42.599431,46.301228,41.877851,53.733927,51.939510,40.040731,35.319192
1980-09-01,29.683738,49.537699,34.538666,75.030947,40.806981,44.548013,56.670426,41.102359,51.371028,48.486300,...,29.700057,42.049358,55.529272,43.558495,47.102652,42.793169,54.871391,52.882659,41.000764,36.544604
1980-12-01,30.814116,51.739704,35.421798,77.933464,42.321892,45.575306,57.706160,42.539525,52.985650,49.618099,...,30.708274,43.447618,57.046816,45.110730,48.629228,44.197269,56.805299,54.104479,42.128805,37.006564
1981-03-01,31.721117,53.684943,36.325785,80.374103,43.673999,46.758200,58.726335,43.721310,54.788774,50.729013,...,31.478240,44.622582,58.477071,46.688371,49.956587,45.437419,58.226981,55.446501,43.160182,37.614947


In [6]:
def __agregacion_y_stack(df, col_name):
    df['año'] = df.index.year
    df = df.groupby('año').mean()
    df = df.stack().reset_index()
    df.columns = ['año','nom_agee',col_name]
    return df

In [7]:
desocupacion = __agregacion_y_stack(desocupacion, 'desocupacion')
itaee = __agregacion_y_stack(itaee, 'itaee')

In [8]:
desocupacion.head()

,año,nom_agee,desocupacion
0,2005,Aguascalientes,0.048516
1,2005,Baja California,0.014249
2,2005,Baja California Sur,0.023082
3,2005,Campeche,0.024602
4,2005,Coahuila de Zaragoza,0.045114


In [9]:
itaee.head()

,año,nom_agee,itaee
0,1980,Aguascalientes,29.539097
1,1980,Baja California,49.406178
2,1980,Baja California Sur,34.272666
3,1980,Campeche,75.241433
4,1980,Coahuila de Zaragoza,40.694501


In [10]:
base_entidades = desocupacion.merge(itaee)
base_entidades.head()

,año,nom_agee,desocupacion,itaee
0,2005,Aguascalientes,0.048516,75.011482
1,2005,Baja California,0.014249,92.976935
2,2005,Baja California Sur,0.023082,75.994203
3,2005,Campeche,0.024602,144.023500
4,2005,Coahuila de Zaragoza,0.045114,85.301155


In [11]:
base_entidades = base_entidades.merge(entidades[['nom_agee','cve_agee']], 'left')

#### Datos externos

##### seguridad pública

In [12]:
dfs = list()
for i in range(2016,2021):
    path = './bases/efipem_estatal_csv/conjunto_de_datos/efipem_estatal_anual_tr_cifra_{}.csv'.format(str(i))
    df = pd.read_csv(path, encoding='utf8')
    df['año'] = str(i)
    dfs.append(df)

In [13]:
egresos = pd.concat(dfs, axis=0)
egresos.head()

,PROD_EST,COBERTURA,ANIO,ID_ENTIDAD,TEMA,CATEGORIA,DESCRIPCION_CATEGORIA,VALOR,ESTATUS,año
0,Estadística de Finanzas Públicas Estatales y M...,Estatal,2016,1,Egresos,Tema,Total de egresos,24431871000,Cifras Definitivas.,2016
1,Estadística de Finanzas Públicas Estatales y M...,Estatal,2016,1,Egresos,Capítulo,Servicios personales,1132656504,Cifras Definitivas.,2016
2,Estadística de Finanzas Públicas Estatales y M...,Estatal,2016,1,Egresos,Concepto,Remuneraciones al personal,565443106,Cifras Definitivas.,2016
3,Estadística de Finanzas Públicas Estatales y M...,Estatal,2016,1,Egresos,Partida Genérica,Despacho del ejecutivo estatal,24876270,Cifras Definitivas.,2016
4,Estadística de Finanzas Públicas Estatales y M...,Estatal,2016,1,Egresos,Partida Genérica,Secretaría general de gobierno,64120732,Cifras Definitivas.,2016


In [14]:
egresos['cve_agee'] = egresos.ID_ENTIDAD.astype(str).apply(lambda x: '0{}'.format(x) if len(x) == 1 else x)

In [15]:
egresos.ID_ENTIDAD.unique() # en esta base falta cdmx

array([ 1,  2,  3,  4,  5,  6,  7,  8, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32],
      dtype=int64)

In [16]:
egresos.DESCRIPCION_CATEGORIA = egresos.DESCRIPCION_CATEGORIA.str.lower()

In [17]:
egresos_totales = egresos[egresos.DESCRIPCION_CATEGORIA == 'total de egresos'][['cve_agee','año','VALOR']]

In [18]:
egresos.CATEGORIA.unique()

array(['Tema', 'Capítulo', 'Concepto', 'Partida Genérica',
       'Subpartida Genérica'], dtype=object)

In [19]:
egresos_seguridad = egresos[(egresos.DESCRIPCION_CATEGORIA.str.match('.*seguridad pública.*')) & 
                            (egresos.CATEGORIA == 'Partida Genérica')]

In [20]:
egresos_seguridad = egresos_seguridad.groupby(['cve_agee','año']).VALOR.sum().reset_index()

In [21]:
egresos_seguridad.columns = ['cve_agee','año','seguridad']
egresos_totales.columns = ['cve_agee','año','total']
egresos_seguridad = egresos_seguridad.merge(egresos_totales)

In [22]:
egresos_seguridad['pct_seguridad'] = egresos_seguridad.seguridad/egresos_seguridad.total

In [23]:
egresos_seguridad.año = egresos_seguridad.año.astype(int)

In [24]:
# agregamos cdmx a la base
cdmx = pd.read_csv('./bases/cdmx_egresos.csv')
cdmx.cve_agee = cdmx.cve_agee.astype(str).apply(lambda x: '0{}'.format(x))
cdmx

,cve_agee,año,seguridad,total,pct_seguridad
0,09,2016,"585,598,441","210,844,683,185",0.002777
1,09,2017,"499,977,602","218,651,533,100",0.002287
2,09,2018,"524,096,536","232,997,023,500",0.002249
3,09,2019,483534119.5,"230,640,525,639",0.002096
4,09,2020,"442,971,703","219,374,758,023",0.002019


In [25]:
egresos_seguridad = pd.concat([egresos_seguridad, cdmx], axis=0)

In [26]:
base_entidades = base_entidades.merge(egresos_seguridad[['cve_agee','pct_seguridad','año']], 'left')

In [27]:
base_entidades = base_entidades.dropna(axis=0)

In [28]:
base_entidades

,año,nom_agee,desocupacion,itaee,cve_agee,pct_seguridad
352,2016,Aguascalientes,0.039734,122.457200,01,0.009233
353,2016,Baja California,0.026309,115.051769,02,0.011374
354,2016,Baja California Sur,0.045898,116.273239,03,0.005872
355,2016,Campeche,0.036902,83.292750,04,0.016885
356,2016,Coahuila de Zaragoza,0.044218,106.099553,05,0.009588
...,...,...,...,...,...,...
507,2020,Tamaulipas,0.034233,99.796118,28,0.010747
508,2020,Tlaxcala,0.053416,103.729965,29,0.013439
509,2020,Veracruz de Ignacio de la Llave,0.029332,94.261150,30,0.010747
510,2020,Yucatán,0.027387,112.759781,31,0.019302


##### gini y pobreza

In [29]:
#! pip install openpyxl

In [30]:
gini_pob = pd.read_excel('./bases/GINI1620.xlsx')
gini_pob.head()

,ESTADO,CVE_ENT,GINI16,GINI17,GINI18,GINI19,GINI20,POB16,POB17,POB18,POB19,POB20
0,AGUASCALIENTES,1.0,0.384828,0.389908,0.394987,0.385318,0.375648,28.946480,27.605944,26.265408,26.946050,27.626693
1,BAJA CALIFORNIA,2.0,0.402005,0.390181,0.378356,0.388014,0.397671,22.604867,23.101777,23.598687,23.054402,22.510117
2,BAJA CALIFORNIA SUR,3.0,0.385390,0.383677,0.381964,0.383870,0.385776,22.880050,20.726107,18.572164,23.087194,27.602224
3,CAMPECHE,4.0,0.425516,0.432903,0.440291,0.435528,0.430766,45.660192,47.312244,48.964296,49.756721,50.549147
4,COAHUILA DE ZARAGOZA,5.0,0.384922,0.388852,0.392782,0.386012,0.379241,27.055713,26.281709,25.507705,25.562373,25.617041


In [31]:
gini = gini_pob.iloc[:,2:7].transpose()
pobreza = gini_pob.iloc[:,7:].transpose()

In [32]:
gini.columns = entidades.cve_agee
pobreza.columns = entidades.cve_agee

In [33]:
gini.index = [2016,2017,2018,2019,2020]
pobreza.index = [2016,2017,2018,2019,2020]

In [34]:
gini = gini.stack().reset_index()
gini.columns = ['año','cve_agee','gini']
gini.head()

,año,cve_agee,gini
0,2016,01,0.384828
1,2016,02,0.402005
2,2016,03,0.385390
3,2016,04,0.425516
4,2016,05,0.384922


In [35]:
pobreza = pobreza.stack().reset_index()
pobreza.columns = ['año','cve_agee','pobreza']
pobreza.head()

,año,cve_agee,pobreza
0,2016,01,28.946480
1,2016,02,22.604867
2,2016,03,22.880050
3,2016,04,45.660192
4,2016,05,27.055713


In [36]:
base_entidades = base_entidades.merge(gini)
base_entidades = base_entidades.merge(pobreza)

In [37]:
base_entidades

,año,nom_agee,desocupacion,itaee,cve_agee,pct_seguridad,gini,pobreza
0,2016,Aguascalientes,0.039734,122.457200,01,0.009233,0.384828,28.946480
1,2016,Baja California,0.026309,115.051769,02,0.011374,0.402005,22.604867
2,2016,Baja California Sur,0.045898,116.273239,03,0.005872,0.385390,22.880050
3,2016,Campeche,0.036902,83.292750,04,0.016885,0.425516,45.660192
4,2016,Coahuila de Zaragoza,0.044218,106.099553,05,0.009588,0.384922,27.055713
...,...,...,...,...,...,...,...,...
155,2020,Tamaulipas,0.034233,99.796118,28,0.010747,0.398158,34.942460
156,2020,Tlaxcala,0.053416,103.729965,29,0.013439,0.378423,59.287522
157,2020,Veracruz de Ignacio de la Llave,0.029332,94.261150,30,0.010747,0.387534,58.603154
158,2020,Yucatán,0.027387,112.759781,31,0.019302,0.436650,49.485626


In [38]:
base_entidades.to_csv('base_entidades.csv', index = False)

### Base Final

In [14]:
base_individual = pd.read_csv('base_individual.csv')
base_individual.head()

,año,cve_agee,renglon,hogar,viv_sel,upm,perc_seguridad,bienestar_econ,seguridad,falta_alumbrado,...,escolaridad,barrera_vivienda,casa_independiente,departamento,vecindad,cuarto_azotea,local_no_habitacional,con_trabajo,no_puede_trabajar,no_trabajo
0,2015,1,1,1,1,100006,0,1,0,1,...,6,0,1,0,0,0,0,1,0,0
1,2015,1,2,1,2,100006,0,1,1,0,...,6,0,1,0,0,0,0,0,1,0
2,2015,1,1,1,3,100006,1,1,0,0,...,2,0,1,0,0,0,0,0,1,0
3,2015,1,2,1,4,100006,1,1,1,0,...,8,0,1,0,0,0,0,1,0,0
4,2015,1,1,1,5,100006,1,1,0,0,...,3,1,1,0,0,0,0,1,0,0


In [15]:
base_entidades = pd.read_csv('base_entidades.csv')
base_entidades.head()

,año,nom_agee,desocupacion,itaee,cve_agee,pct_seguridad,gini,pobreza
0,2016,Aguascalientes,0.039734,122.457200,1,0.009233,0.384828,28.946480
1,2016,Baja California,0.026309,115.051769,2,0.011374,0.402005,22.604867
2,2016,Baja California Sur,0.045898,116.273239,3,0.005872,0.385390,22.880050
3,2016,Campeche,0.036902,83.292750,4,0.016885,0.425516,45.660192
4,2016,Coahuila de Zaragoza,0.044218,106.099553,5,0.009588,0.384922,27.055713


In [16]:
base_entidades = base_entidades.drop('nom_agee',axis=1)

In [17]:
base_individual = base_individual[base_individual.año.between(2016,2020)]

In [18]:
base_final = base_individual.merge(base_entidades, 'left')

In [19]:
base_final

,año,cve_agee,renglon,hogar,viv_sel,upm,perc_seguridad,bienestar_econ,seguridad,falta_alumbrado,...,cuarto_azotea,local_no_habitacional,con_trabajo,no_puede_trabajar,no_trabajo,desocupacion,itaee,pct_seguridad,gini,pobreza
0,2016,1,3,1,1,100006,1,1,1,0,...,0,0,0,1,0,0.039734,122.457200,0.009233,0.384828,28.946480
1,2016,1,1,1,2,100006,1,0,1,0,...,0,0,1,0,0,0.039734,122.457200,0.009233,0.384828,28.946480
2,2016,1,4,1,3,100006,1,1,0,0,...,0,0,1,0,0,0.039734,122.457200,0.009233,0.384828,28.946480
3,2016,1,1,1,4,100006,1,1,0,0,...,0,0,1,0,0,0.039734,122.457200,0.009233,0.384828,28.946480
4,2016,1,1,2,4,100006,1,1,0,1,...,0,0,1,0,0,0.039734,122.457200,0.009233,0.384828,28.946480
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451621,2020,20,2,1,7,2061490,1,1,0,0,...,0,0,0,1,0,0.021553,97.015056,0.011449,0.419259,61.664248
451622,2020,20,1,1,8,2061490,1,1,1,0,...,0,0,1,0,0,0.021553,97.015056,0.011449,0.419259,61.664248
451623,2020,20,1,1,10,2061490,1,1,0,0,...,0,0,1,0,0,0.021553,97.015056,0.011449,0.419259,61.664248
451624,2020,20,2,1,11,2061490,1,1,1,3,...,0,0,1,0,0,0.021553,97.015056,0.011449,0.419259,61.664248


In [26]:
base_final = base_final[(base_final.perc_seguridad != 9) & 
                        (~base_final.falta_agua.isin([9,3])) &
                        (~base_final.falta_alumbrado.isin([9,3])) &
                        (~base_final.baches_fugas.isin([9,3]))]

In [28]:
base_final.shape

(442965, 29)

In [29]:
base_final.to_csv('base_final.csv', index=False)